<a href="https://colab.research.google.com/github/rahiakela/computer-vision-research-and-practice/blob/main/hands-on-computer-vision-with-detectron2/04-custom-object-detection/01_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Processing

The dataset used is the brain tumor object detection dataset available from [Kaggle](https://www.kaggle.com/datasets/davidbroberts/brain-tumor-object-detectiondatasets).

This dataset is chosen because medical image processing is a critical subfield in computer vision. At the same
time, the task is challenging, and the number of images is appropriate for demonstration purposes.



##Setup

In [ ]:
!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"
!sudo apt-get install tree
!pip install -q pylabel

In [ ]:
import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer

import os
from glob import glob
import shutil
from tqdm import tqdm
import yaml
from yaml.loader import SafeLoader
from pylabel import importer

import torch
import cv2

import numpy as np
import matplotlib.pyplot as plt

# Suppress some user warnings
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
print(detectron2.__version__)

0.6


In [ ]:
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/datasets/braintumors.zip?raw=true -O braintumors.zip
!unzip braintumors.zip -d braintumors

##Dataset

In [ ]:
data_folder = "braintumors"
data_folder_yolo = data_folder + "_yolo"
data_folder_coco = data_folder + "_coco"

folders = os.listdir("braintumors")
print(folders)

['coronal_t1wce_2_class', 'sagittal_t1wce_2_class', 'axial_t1wce_2_class']


In [ ]:
!tree braintumors/ -d

braintumors/
├── axial_t1wce_2_class
│   ├── images
│   │   ├── test
│   │   └── train
│   └── labels
│       ├── test
│       └── train
├── coronal_t1wce_2_class
│   ├── images
│   │   ├── test
│   │   └── train
│   └── labels
│       ├── test
│       └── train
└── sagittal_t1wce_2_class
    ├── images
    │   ├── test
    │   └── train
    └── labels
        ├── test
        └── train

21 directories


In [ ]:
# let's count the number of images and labels from a dataset with the YOLO annotation format


##Input image

In [ ]:
img = cv2.imread("input.jpeg")
height, width = img.shape[:2]
image = torch.as_tensor(img.astype("float32").transpose(2, 0, 1))

In [ ]:
image

tensor([[[199., 199., 199.,  ..., 203., 203., 203.],
         [199., 199., 199.,  ..., 203., 203., 203.],
         [199., 199., 199.,  ..., 204., 204., 204.],
         ...,
         [ 73.,  90.,  91.,  ...,  95.,  92.,  91.],
         [ 87.,  88.,  86.,  ...,  92.,  93.,  93.],
         [ 81.,  78.,  84.,  ...,  83.,  89.,  90.]],

        [[147., 147., 147.,  ..., 144., 144., 144.],
         [147., 147., 147.,  ..., 144., 144., 144.],
         [147., 147., 147.,  ..., 145., 145., 145.],
         ...,
         [114., 131., 133.,  ..., 137., 134., 133.],
         [131., 132., 130.,  ..., 133., 134., 134.],
         [125., 122., 128.,  ..., 124., 127., 128.]],

        [[106., 106., 106.,  ..., 104., 104., 104.],
         [106., 106., 106.,  ..., 104., 104., 104.],
         [106., 106., 106.,  ..., 105., 105., 105.],
         ...,
         [206., 223., 222.,  ..., 233., 230., 229.],
         [222., 223., 221.,  ..., 226., 227., 227.],
         [216., 213., 219.,  ..., 217., 221., 222.]]]